In [1]:
from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.simplefilter("ignore", ResourceWarning)

# Vectrix Demo 👨🏻‍💻
This notebook demonstrates the functions for importing data from various sources. 
Loading it into a VectorStore, and then using it to answer questions with a Retrieval Augemented Reasoning  🦜🔗 LangGraph.

## Creating a new project



In [4]:
from vectrix import DB
db = DB()

print(db.list_projects())
db.create_project("Test", description="This is a test project")

['Vectrix', 'Schoups']


12

In [3]:
db.remove_project("Test")

True

## Importing Data
### 1. From a URL 🔗

**Web Crawling and Data Extraction Example**


In [ ]:
from vectrix.importers import WebScraper

scraper = WebScraper(project_name='Test')
scraper.get_all_links("https://vectrix.ai/")

In [5]:
from vectrix.importers import WebScraper

scraper = WebScraper(project_name='Test')
scraper.full_import("https://dataframe.be", max_pages=100)

Added links to confirm for base URL: https://dataframe.be
Added links to confirm for base URL: https://dataframe.be
Added links to confirm for base URL: https://dataframe.be
Added links to confirm for base URL: https://dataframe.be
Added links to confirm for base URL: https://dataframe.be
Added links to confirm for base URL: https://dataframe.be
Added links to confirm for base URL: https://dataframe.be
Added links to confirm for base URL: https://dataframe.be
Added links to confirm for base URL: https://dataframe.be
Added links to confirm for base URL: https://dataframe.be


In [ ]:
print(db.get_links_to_confirm("Test"))

In [ ]:
db.add_links_to_index(links=["https://langchain-ai.github.io/langgraph/reference/graphs/"],base_url="https://langchain-ai.github.io/langgraph/reference/graphs/", project_name="Test", status="Confirm page import")

In [ ]:
print(db.get_indexing_status("Test"))

In [ ]:
confirmed_links = [link['url'] for link in db.get_links_to_confirm("Test")]
scraper.download_pages(confirmed_links, project_name="Test")

### 2. Upload files ⬆️
You can also upload files and add them to the vector store, Vectrix will automaticly detect the file type extract the text and chunk the content into blocks.

In [ ]:
from vectrix.importers import Files
files = Files()

files.upload_file(
    document_paths=["./files/pdf_with_scannedtext.pdf"],
    project_name="Test"
)

In [ ]:
files = db.list_files("Test")
print(files)

In [ ]:
db.remove_file(project_name="Test", file_name="pdf_with_scannedtext.pdf")

In [ ]:
from langchain_community.document_loaders import SpiderLoader

loader = SpiderLoader(
    url="https://dataframe.be",
    mode="crawl",
    params={
        'limit': 100,
        'metadata': True
        },
)

data = loader.load()


In [ ]:
from trafilatura import extract

result = extract(data[0].page_content)


In [ ]:
print(data[5].metadata)